# Bee Classification with Keras - Training Images


This script goes along the blog post

"Building powerful image classification models using very little data"

from blog.keras.io.

In our setup, we:

- created a data/ folder under the root
- created train/ and validation/ subfolders inside data/
- created bumblebee/ and honeybee/ subfolders inside train/ and validation/
- put the bumblebee pictures 10000-13034 in data/train/bumblebee
- put the bumblebee pictures 13035-13968 in data/validation/bumblebee
- put the honeybee pictures 10005-12772 in data/train/honeybee
- put the honeybee pictures 12776-13966 in data/validation/honeybee
- put the test images in test/ folder under the root


In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras import backend as K

# dimensions of our images.
img_width, img_height = 200, 200

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 4128
nb_validation_samples = 1008
epochs = 50
batch_size = 16
early_stopping = EarlyStopping(monitor='val_loss', patience=4)

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[early_stopping])

model.save_weights('first_try.h5')
model.save('first_try_model.h5')
print('Model saved to first_try_model.h5 file!')

Using TensorFlow backend.


Found 4128 images belonging to 2 classes.
Found 1008 images belonging to 2 classes.
Epoch 1/50
258/258 [==============================] - 410s 2s/step - loss: 0.7331 - acc: 0.6105 - val_loss: 0.6683 - val_acc: 0.5972
Epoch 2/50
258/258 [==============================] - 410s 2s/step - loss: 0.6692 - acc: 0.6962 - val_loss: 0.5125 - val_acc: 0.7778
Epoch 3/50
258/258 [==============================] - 409s 2s/step - loss: 0.6146 - acc: 0.7132 - val_loss: 1.0385 - val_acc: 0.6419
Epoch 4/50
258/258 [==============================] - 411s 2s/step - loss: 0.5707 - acc: 0.7284 - val_loss: 0.5315 - val_acc: 0.7579
Epoch 5/50
258/258 [==============================] - 407s 2s/step - loss: 0.5595 - acc: 0.7401 - val_loss: 0.4967 - val_acc: 0.8065
Epoch 6/50
258/258 [==============================] - 409s 2s/step - loss: 0.5420 - acc: 0.7548 - val_loss: 0.4800 - val_acc: 0.7887
Epoch 7/50
258/258 [==============================] - 409s 2s/step - loss: 0.5379 - acc: 0.7570 - val_loss: 0.4665 - v

### Bottleneck features

In [4]:
import numpy as np
import math
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.models import load_model
from keras.callbacks import EarlyStopping

# dimensions of our images.
img_width, img_height = 200, 200

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
#find the equal numbers for tain and validation samples to be divided by batch_size, otherwise errors out
nb_train_samples = 4128
nb_validation_samples = 1008
epochs = 50
batch_size = 16
early_stopping = EarlyStopping(monitor='val_loss', patience=4)

def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    #bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    #changed to following two lines as original gives error, testing with predict_size_train
    # predict_size_train = int(math.ceil(nb_train_samples // batch_size))
    predict_size_train = nb_train_samples // batch_size
    # bottleneck_features_train = model.predict_generator(generator, predict_size_train)
    bottleneck_features_train = model.predict_generator(generator, steps=predict_size_train)
    #added b argument as Numpy gives error on Windows if omitted
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    # bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)
    #changed to following two lines as original gives error
    # predict_size_validation = int(math.ceil(nb_validation_samples // batch_size))
    predict_size_validation = nb_validation_samples // batch_size
    bottleneck_features_validation = model.predict_generator(generator, predict_size_validation)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

#added ,"rb" argument
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb')) 
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),
              callbacks=[early_stopping])
    
    model.save_weights(top_model_weights_path)


save_bottleneck_features()
train_top_model()

model.save('top_model.h5')
print('Model saved to top_model.h5 file!')

Found 4128 images belonging to 2 classes.
Found 1008 images belonging to 2 classes.
Train on 4128 samples, validate on 1008 samples
Epoch 1/50
4128/4128 [==============================] - 26s 6ms/step - loss: 1.1511 - acc: 0.5913 - val_loss: 0.5868 - val_acc: 0.6637
Epoch 2/50
4128/4128 [==============================] - 26s 6ms/step - loss: 0.6237 - acc: 0.6734 - val_loss: 0.5372 - val_acc: 0.7063
Epoch 3/50
4128/4128 [==============================] - 26s 6ms/step - loss: 0.5425 - acc: 0.7422 - val_loss: 0.4609 - val_acc: 0.7857
Epoch 4/50
4128/4128 [==============================] - 26s 6ms/step - loss: 0.4999 - acc: 0.7631 - val_loss: 0.4598 - val_acc: 0.8065
Epoch 5/50
4128/4128 [==============================] - 27s 6ms/step - loss: 0.4594 - acc: 0.7989 - val_loss: 0.5803 - val_acc: 0.7421
Epoch 6/50
4128/4128 [==============================] - 27s 6ms/step - loss: 0.4087 - acc: 0.8154 - val_loss: 0.5595 - val_acc: 0.7381
Epoch 7/50
4128/4128 [==============================] - 27

### Fine-tuning

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras import backend as K


# path to the model weights files.
# weights_path = 'vgg16_weights.h5'
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'


# dimensions of our images.
img_width, img_height = 200, 200

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 4128
nb_validation_samples = 1008
epochs = 50
batch_size = 16
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# added if - else as errored out about input_shape
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


# build the VGG16 network
#added input_shape, changed model to base_model
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape) 
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# the original did not work, commented out
# model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
# changed 25 to 15, also try 11
for layer in model.layers[:11]: 
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate, changed from 1e-4 to 1e-3
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), 
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
#changed from steps_per_epoch=nb_train_samples, also validation_samples, to // batch_size
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[early_stopping])

model.save('final_model.h5')
print('Model saved to final_model.h5 file!')

Using TensorFlow backend.


Model loaded.
Found 4128 images belonging to 2 classes.
Found 1008 images belonging to 2 classes.
Epoch 1/50
258/258 [==============================] - 21622s 84s/step - loss: 0.4424 - acc: 0.7987 - val_loss: 0.4209 - val_acc: 0.8085
Epoch 2/50
258/258 [==============================] - 21282s 82s/step - loss: 0.3104 - acc: 0.8798 - val_loss: 0.3128 - val_acc: 0.8661
Epoch 3/50
258/258 [==============================] - 21268s 82s/step - loss: 0.2534 - acc: 0.9009 - val_loss: 0.3046 - val_acc: 0.8829
Epoch 4/50
258/258 [==============================] - 21151s 82s/step - loss: 0.2142 - acc: 0.9172 - val_loss: 0.2795 - val_acc: 0.8958
Epoch 5/50
258/258 [==============================] - 21186s 82s/step - loss: 0.1672 - acc: 0.9409 - val_loss: 0.2874 - val_acc: 0.8988
Epoch 6/50
258/258 [==============================] - 21302s 83s/step - loss: 0.1679 - acc: 0.9344 - val_loss: 0.2186 - val_acc: 0.9127
Epoch 7/50
258/258 [==============================] - 21170s 82s/step - loss: 0.1351 -

### Pre-processing test images and Predictions

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.models import load_model
from PIL import Image
import numpy as np
import os
import csv

target_size = (200,200)
# model = load_model("first_try_model.h5")
# model = load_model("top_model.h5")
model = load_model("final_model.h5")

def predict(model,img,target_size):
    if img.size != target_size:
        img = img.resize(target_size)
        
    x = img.getdata()
    x = np.expand_dims(x,axis=0)
    x = (x/255.)
    x = x.reshape(1,200,200,3)
    preds = model.predict(x)
    return preds[0]


#for some reason range needs to be +1 greater than last image?
fnames = ["test/{}.jpg".format(i) for i in range(20000,20992)] 
I = 1
predictions = []

with open("predictions.csv","w") as f:
        f.write("filename,prediction\n")

for fname in fnames:
    img = Image.open(fname)
    preds = predict(model,img,target_size)
    predictions.append(preds)
    for pred in preds:
        out=csv.writer(open("predictions.csv","a"),delimiter=',',quoting=csv.QUOTE_ALL)
        out.writerow(fname)
        out.writerow(preds)
    I += 1
    
print("Predictions in Predictions.csv file.")

out=csv.writer(open("predictions_short.csv","w"),delimiter=',',quoting=csv.QUOTE_ALL)
out.writerows(predictions)


Predictions in Predictions.csv file.


In [3]:
out=csv.writer(open("predictions_short.csv","w"),delimiter=',',quoting=csv.QUOTE_ALL)
out.writerows(predictions)